In [1]:
import pandas as pd
import datetime
from datetime import date, time, timedelta
import itertools
import time
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Global variables (CAPS)

battery_power = 300
battery_cap = 580
charge_eff = 90
discharge_eff = 90
mlf = 0.991

In [4]:
def raw_power(charge_forecast, discharge_forecast, opening_cap):
    if charge_forecast == 1 and discharge_forecast == 0:
        return -min(battery_power, (battery_cap - opening_cap)/(charge_eff/100)*2)
    elif charge_forecast == 0 and discharge_forecast == 1:
        return min(battery_power, (opening_cap/(discharge_eff/100))*2 )
    else:
        return 0

In [5]:
def market_dispatch(raw_power):
    if raw_power < 0:
        return (raw_power/2)
    elif raw_power > 0:
        return (raw_power/2)*discharge_eff/100
    else:
        return 0

In [6]:
def market_revenue(market_dispatch,spot_price):
    if market_dispatch < 0:
        return market_dispatch*spot_price*(1/mlf)
    elif market_dispatch > 0:
        return market_dispatch*spot_price*mlf
    else:
        return 0

In [7]:
def closing_capacity(market_dispatch, opening_cap):
    if market_dispatch < 0:
        x = opening_cap - market_dispatch * (charge_eff/100)
        return (max(0, min(round(x), battery_cap)))
    elif market_dispatch > 0:
        x = opening_cap - market_dispatch * (100/discharge_eff)
        return (max(0, min(round(x), battery_cap)))
    else:
        x = opening_cap - market_dispatch * (100/discharge_eff)
        return (max(0, min(round(x), battery_cap)))

In [8]:
def future_prices(index, subsample):
    past_avg = 0
    future_avg = 0
    if index < 10:   # if number of past prices less than 10
        future_avg = subsample.iloc[index+1:index + 11]['Spot Price'].mean()
    elif index >= (len(subsample) - 10):   # if number of future prices less than 10
        future_avg = subsample.iloc[index+1:len(subsample)]['Spot Price'].mean()
    else:
        future_avg = subsample.iloc[index+1:index + 11]['Spot Price'].mean()
    return future_avg

In [30]:
def charging_condition(index, current_price, future_price):
    ''' Determine whether current period is forcasted to charge or discharge '''
    
    comparison_threshold = 8
    current_future_diff = abs(future_price - current_price)
    
    # Discharge conditions
    if (current_price > future_price):
        if current_future_diff >= comparison_threshold:
            subsample["Discharge Forecast"].values[index] = 1
    
    # Charge conditions
    if (current_price < future_price):
        if current_future_diff >= comparison_threshold:
            subsample["Charge Forecast"].values[index] = 1

In [31]:
df = pd.read_excel("../../data/market_data.xlsx") 

In [32]:
vic_spotprice = df.filter(items=['Time (UTC+10)', 'Regions VIC Trading Price ($/MWh)'])
vic_spotprice = vic_spotprice.rename(columns={'Regions VIC Trading Price ($/MWh)': 'Spot Price', 'Time (UTC+10)': 'Time'})
# Change column type to datetime type
vic_spotprice['Time'] = pd.to_datetime(vic_spotprice['Time'])

In [33]:
subsample = vic_spotprice.copy()

In [34]:
# Create new columns

subsample["Future Average"] = 0.0
subsample["Charge Forecast"] = 0
subsample["Discharge Forecast"] = 0
subsample["Raw Power"] = 0
subsample["Market Dispatch"] = 0
subsample["Market Revenue"] = 0
subsample["Opening Capacity"] = 0
subsample["Closing Capacity"] = 0

In [35]:
%%time

for index, row in subsample.iterrows():
    future_average = future_prices(index, subsample)
    subsample["Future Average"].values[index] = future_average
    
    current_price = subsample.iloc[index]["Spot Price"]

    charging_condition(index, current_price, future_average)
    
    if index != 0:
        subsample["Opening Capacity"].values[index] = subsample["Closing Capacity"].values[index-1]
    
    subsample["Raw Power"].values[index] = raw_power(subsample["Charge Forecast"].values[index], subsample["Discharge Forecast"].values[index], subsample["Opening Capacity"].values[index])
    subsample["Market Dispatch"].values[index] = market_dispatch(subsample["Raw Power"].values[index])
    subsample["Market Revenue"].values[index] = market_revenue(subsample["Market Dispatch"].values[index], subsample["Spot Price"].values[index])
    subsample["Closing Capacity"].values[index] = closing_capacity(subsample["Market Dispatch"].values[index], subsample["Opening Capacity"].values[index])

CPU times: user 24.8 s, sys: 34.2 ms, total: 24.9 s
Wall time: 24.8 s


In [36]:
# Current > Past  --> Discharge
# Current < Future --> Charge

sum(subsample["Market Revenue"])

121669662

In [37]:
subsample.tail(50)

,Time,Spot Price,Future Average,Charge Forecast,Discharge Forecast,Raw Power,Market Dispatch,Market Revenue,Opening Capacity,Closing Capacity
63407,2021-08-13 23:30:00,76.50,62.892000,0,1,300,135,10234,280,130
63408,2021-08-14 00:00:00,76.38,58.523000,0,1,288,129,9764,130,0
63409,2021-08-14 00:30:00,77.12,55.324000,0,1,0,0,0,0,0
63410,2021-08-14 01:00:00,87.96,51.621000,0,1,0,0,0,0,0
63411,2021-08-14 01:30:00,76.75,49.494000,0,1,0,0,0,0,0
63412,2021-08-14 02:00:00,77.71,47.832000,0,1,0,0,0,0,0
63413,2021-08-14 02:30:00,70.46,47.048000,0,1,0,0,0,0,0
63414,2021-08-14 03:00:00,50.07,48.060000,0,0,0,0,0,0,0
63415,2021-08-14 03:30:00,46.53,47.890000,0,0,0,0,0,0,0
63416,2021-08-14 04:00:00,37.60,45.958000,1,0,-300,-150,-5691,0,135


In [18]:
output = subsample[['Time', 'Raw Power', 'Opening Capacity']].rename(columns={"Time": "datetime", "Raw Power": "power", "Opening Capacity":"capacity"})

In [19]:
output.to_csv("test_submission.csv", index=False)